In [ ]:
!pip install openai;
!pip install anthropic;
!pip install chromadb;
!pip install transformers;
!pip install huggingface_hub;
!pip install --upgrade huggingface_hub;

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import anthropic
import openai
import sklearn
import torch
import chromadb
import json
from collections import defaultdict

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
token = #huggingface token

In [ ]:
import openai
import os
from openai import OpenAI

openai_key =  #your key
openai.api_key = openai_key
os.environ["OPENAI_API_KEY"] = openai_key


def query_gpt_chat(api_key, messages, model="gpt-3.5-turbo-0125", temperature=0.2,max_tokens =1024):
    openai.api_key = api_key
    client = OpenAI(
    api_key=api_key,
    )

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens =max_tokens,
        temperature = temperature
    )
    return response.choices[0].message.content

In [ ]:
import re
deepseek_key = #your deepseek key
os.environ["deepseek_key"] = deepseek_key
def query_deepseek(api_key, messages, model="deepseek-ai/deepseek-r1",temperature=0.2,max_tokens =1024):
    
    client = OpenAI(
        api_key=api_key,
        base_url= "https://integrate.api.nvidia.com/v1",
        )
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens =max_tokens,
        temperature = temperature,
        stream=False
    )
    response_text = response.choices[0].message.content
    return re.sub(r"<think>.*?</think>\n*", "", response_text, flags=re.DOTALL).strip()

In [ ]:
ANTHROPIC_API_KEY= #your key
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

def query_claude(api_key, messages_system, messages_user, model="claude-3-5-sonnet-20241022",temperature=0.2,max_tokens =1024):
    client = anthropic.Anthropic(api_key=api_key)
    message = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        system=messages_system,
        messages=messages_user
    )
    return message.content[0].text 

In [ ]:
import time

def run_llm(text,system=None):
    api_key = os.getenv("OPENAI_API_KEY")
    sys_prompt = system 
    msg= [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text}
    ]
    return query_gpt_chat(api_key, msg)

    #deepseek
    # api_key = os.getenv("deepseek_key")
    # return query_deepseek(api_key, msg) 
           
    
    #CLAUD
    # api_key_antropic = os.getenv("ANTHROPIC_API_KEY")
    # return query_claude( api_key=api_key_antropic, messages_system=sys_prompt, messages_user=[{"role": "user", "content": text}])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
import torch
from chromadb.config import Settings
import chromadb


MODEL_NAME = "microsoft/codebert-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()



MASK_TOKEN = "<mask>"
SEPARATOR_TOKEN = "<sep>"
PAD_TOKEN = "<pad>"
CLS_TOKEN = "<cls>"

def prepare_tokenizer(tokenizer):
    tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
    tokenizer.add_special_tokens({"sep_token": SEPARATOR_TOKEN})
    tokenizer.add_special_tokens({"cls_token": CLS_TOKEN})
    tokenizer.add_special_tokens({"mask_token": MASK_TOKEN})
    return tokenizer

tokenizer = prepare_tokenizer(tokenizer)
def get_embeddings(texts):
    inputs = tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

In [ ]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512) 
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs

def save_embeddings(embeddings, file_path):
    np.save(file_path, embeddings)

In [ ]:
def find_nearest_samples(test_embedding, collection, n_neighbors=20):
    query_results = collection.query(query_embeddings=[test_embedding], n_results=n_neighbors)
    
    documents = query_results['documents']
    metadatas = query_results['metadatas']
    
    nearest_by_class = {}

    for doc, meta, distance in zip(documents[0], metadatas[0], query_results['distances'][0]):
        bug_type = meta['bug_type']
        if bug_type not in nearest_by_class:
            nearest_by_class[bug_type] = {'distance': distance, 'document': doc, 'metadata': meta}
        elif distance < nearest_by_class[bug_type]['distance']:
            nearest_by_class[bug_type] = {'distance': distance, 'document': doc, 'metadata': meta}
    
    nearest_documents = [entry['document'] for entry in nearest_by_class.values()]
    nearest_metadatas = [entry['metadata'] for entry in nearest_by_class.values()]
    return nearest_documents, nearest_metadatas

In [ ]:
X_train= train_df['post']
X_train_summary = summary_with_reasoning #from stage 1
y_train = train_df[cls]

X_test = test_df['post']
y_test = test_df[cls]

In [ ]:
embeddings = get_embeddings(X_train)

In [ ]:
client_chroma = chromadb.Client()
collection_name = "bug_db2"
collection = client_chroma.get_or_create_collection(name=collection_name)
import math


for idx, (embedding, text, label,summary) in enumerate(zip(embeddings, X_train, y_train, X_train_summary)):

    collection.add(
        embeddings = [embedding.tolist()],
        documents=text,
        metadatas=[{'bug_type': label, 'summay':summary}],
        ids=[f"doc_{idx}"]  
    )


In [ ]:
cls_list = list(y_train.unique())
cls_description = bug_type_character #from stage 1
system_prompt =  f"""You are a one-shot classifier that categorizes Stack Overflow posts. You have to return one value from this list only: {cls_list}. Based on the bug type of the test sample, return only ONE VALUE FROM THE LIST. The characteristics of each class are provided below:"""

for class_name in cls_list:

    system_prompt = system_prompt +"\n"+class_name+": "+cls_description[class_name]
    print(len(cls_description[class_name]))

    
print(system_prompt)

In [ ]:
def classify_with_llm(test_sample, nearest_samples,system_prompt):
    prompt = ""
    for neighbor_text, neighbor_metadata in zip(nearest_samples[0], nearest_samples[1]):
        neighbor_label = neighbor_metadata['bug_type']

        neighbor_summary = neighbor_metadata['summay']
        prompt += f"Example Post: {neighbor_summary}. <class>: {neighbor_label}\n"

        
    prompt += f"\nTest Sample: {test_sample}. <class>?:"
    
    
    result = run_llm(prompt,system_prompt)
    return result

In [ ]:
def simple_summary(post):
    system_prompt="You are given a Stack Overflow post. You have to summarize the post. You may illustrate the error in detail. Do not provide any solution to this problem. The overall output should be short. Just return the summary without extra words."
    client_prompt = f"<Post> {post} </Post>"
    return run_llm(client_prompt,system_prompt)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix
import numpy as np

# true_labels = y_test.tolist()
predictions_bt = []
not_found_count = 0
not_founds = [ ]
for i in range(len(X_test)):#
    test_embedding = get_embeddings(pd.Series([X_test.iloc[i]]))[0]
    nearest_documents, nearest_metadatas = find_nearest_samples(test_embedding, collection)
    X_test_summary = simple_summary(X_test.iloc[i]]) #
    classification = classify_with_llm(X_test_summary, [nearest_documents, nearest_metadatas],system_prompt)
    
    class_found = False
    for class_name in y_train.unique():
        if class_name.lower().strip() in classification.lower().strip():
            predictions_bt.append(class_name)
            print(f"{i}, pred: {class_name}")
            class_found = True
            break
    
    if not class_found:
        print(f"not found: {classification}")
        predictions_bt.append(classification)
        not_founds.append(i)
        not_found_count += 1

In [ ]:
#execute this code only if LLM fails to make a prediction from the class list

not_founds_2=[]
for i in not_founds:
    test_embedding = get_embeddings(pd.Series([X_test.iloc[i]]))[0]
    nearest_documents, nearest_metadatas = find_nearest_samples(test_embedding, collection)
    classification = classify_with_llm(X_test_summary.iloc[i], [nearest_documents, nearest_metadatas],system_prompt)
    
    class_found = False
    for class_name in y_test.unique():
        if class_name.strip().lower() in classification.strip().lower():
            predictions_rc[i] =  class_name
            print(f"{i}, class: {class_name}\n\n")
            class_found = True
            break
    
    if not class_found:
        print(f"not found: {classification}")
        not_founds_2.append(i)
        not_found_count += 1